# Audit Logs with Databricks Unity Catalog System Tables

Databricks tracks all operations across our Lakehouse. This is key for governance and being able to analyze your Lakehouse usage.

This is made possible with the `system.access.audit` table. 

This table contains all informations to answer questions such as:

- All operation executed in your account and workspaces
- Which operations are executed against your Unity Catalog objects
- Who is accessing what
- Who and When a table was deleted
- Monitor critical table access
- ...

## Audit Logs table

The `audit_logs` table has the following columns: 
- version
- event_time
- event_date
- event_id
- workspace_id
- source_ip_address
- user_agent
- session_id
- user_identity
- service_name 
- action_name
- request_id
- request_params
- response
- audit_level
- account_id

Audit logs give you the ability to see what users are doing within Databricks. They allow you to see the actions users have taken, such as editing or triggering a job and when it occured. 

## Query example 

The following queries are some example of audit analysis that you can perform.

Make sure you have read access to the system catalog to be able to run the following queries (by default available to admin metastore).

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=governance&org_id=1444828305810485&notebook=%2F02-audit-logs-tables%2F02-audit-log&demo_name=uc-04-system-tables&event=VIEW&path=%2F_dbdemos%2Fgovernance%2Fuc-04-system-tables%2F02-audit-logs-tables%2F02-audit-log&version=1&user_hash=ddf348067415027dc6243f8b145629a2bf969356f50e695cd9fcf1cc18a6c40d">

In [0]:
select * from system.access.audit

In [0]:
select distinct(service_name) from system.access.audit

In [0]:
select
  count(*) as action_count,
  action_name
from
  system.access.audit
where
  service_name = 'unityCatalog'
group by
  action_name
order by
  action_count desc;

In [0]:
SELECT
  *
FROM
  system.access.audit
WHERE
  service_name = 'unityCatalog'
  AND action_name = "getTable"
LIMIT
  100;

In [0]:
SELECT
  event_time,
  action_name,
  user_identity.email as requester,
  request_params,
  *
FROM
  system.access.audit
WHERE
  action_name IN ('updatePermissions', 'updateSharePermissions')
  AND audit_level = 'ACCOUNT_LEVEL'
  AND service_name = 'unityCatalog'
ORDER BY
  action_name,
  event_time
limit
  1000;

In [0]:
SELECT
  event_time,
  action_name,
  user_identity.email as requester,
  request_params,
  *
FROM
  system.access.audit
WHERE
  action_name IN (
    'createCatalog',
    'deleteCatalog',
    'updateCatalog'
  )
  AND audit_level = 'ACCOUNT_LEVEL'
  AND service_name = 'unityCatalog'
ORDER BY
  action_name,
  event_time

### External Locations 
External locations are a common way to load data from cloud object storage into Databricks tables. They can also be used for writing data out of Databricks for external processes. We want to ensure that data is being used properly and is not exfiltrating outside our doman. Let's see how many external locations are being created over time and which ones are the most used. 

In [0]:
-- created external locations
-- monitor possible data exfilltration
select
  event_time,
  event_date,
  date_format(event_date, 'yyyy-MM') AS year_month,
  user_identity.email as user_email,
  service_name,
  action_name,
  request_id,
  request_params.name as external_location_name,
  request_params.skip_validation,
  request_params.credential_name,
  request_params.url,
  request_params.workspace_id,
  request_params.metastore_id,
  response.status_code
from
  system.access.audit
where
  action_name = 'createExternalLocation'

In [0]:
-- most used external locations
select
  event_time,
  event_date,
  date_format(event_date, 'yyyy-MM') AS year_month,
  user_identity.email as user_email,
  service_name,
  action_name,
  request_id,
  request_params,
  request_params.name_arg as external_location_name,
  request_params.workspace_id,
  request_params.metastore_id,
  response.status_code
from
  system.access.audit
where
  action_name = 'getExternalLocation'

### Users 

An important KPI for any platform team administering Databricks is the number of users that they are supporting. This allows admins to quantify the impact is developer productivity and importance to the organization.  

In [0]:
select
  distinct event_date as `Date`,
  date_format(event_date, 'yyyy-MM') AS year_month,
  workspace_id,
  user_identity.email as user_email
from
  system.access.audit

In [0]:
SELECT
  event_time,
  action_name,
  user_identity.email as requester,
  request_params.targetGroupId as deleted_group_id,
  CASE
    WHEN audit_level = 'ACCOUNT_LEVEL' THEN 'deleted from account console'
    WHEN audit_level = 'WORKSPACE_LEVEL' THEN 'deleted from workspace'
  END AS scenario,
  *
FROM
  system.access.audit
WHERE
  action_name = 'removeGroup' --AND request_params.targetGroupId IS NOT NULL
  AND audit_level IN ('ACCOUNT_LEVEL', 'WORKSPACE_LEVEL')
  AND service_name = 'accounts'
ORDER BY
  audit_level,
  event_time;

In [0]:
-- Metastore admin assigned for UC - when and who(userid) changed
SELECT
  event_time,
  action_name,
  user_identity.email AS requester,
  request_params.`owner` AS new_metastore_owner_admin,
  *
FROM
  system.access.audit
WHERE
  action_name = 'updateMetastore'
  AND request_params.`owner` IS NOT NULL
  AND audit_level = 'ACCOUNT_LEVEL'
  AND service_name = 'unityCatalog'
ORDER BY
  event_time
LIMIT
  100;